In [34]:
import pickle
from kymatio import Scattering1D
import os
import pandas as pd
import librosa
import numpy as np
import torch

In [35]:
# scattering order one
def getsc_new(y,J,Q_num,order):
    """
    this function outputs scattering transform of a time-domain signal.
    """
    N = len(y)
    scattering = Scattering1D(J = J,shape=(N,), Q = Q_num, max_order=order)
    Sy = scattering(torch.Tensor(y))
    return Sy

In [36]:
df_val = pd.read_csv("./val_param.csv")
val_id = df_val.values[:,0] 
val_param = df_val.values[:,1:-1]

In [41]:
J = 8
Q = 1
order = 2
val_path = "/scratch/hh2263/drum_data/val/"
num_val = len(val_id)
for i,val in enumerate(val_id):
    filename = val_path + str(val) + "_sound.wav"
    x,fs = librosa.load(filename)
    Sy = getsc_new(x,J,Q,order).T #scattering coefficients
    Sy = Sy.reshape((Sy.shape[0],Sy.shape[1],1))
    y = val_param[i,:] #ground truth
    if i == 0:
        val_input = Sy
        val_gt = y
    else:
        val_input = np.dstack((val_input,Sy))
        val_gt = np.vstack((val_gt,y))
    #print(val_input.shape,val_gt.shape)
    #if i == 2:
    #    break
    
    

KeyboardInterrupt: 

In [47]:
val_input.shape

(128, 43, 1506)

In [48]:
#put all validation set into pickle file
output = open('/scratch/hh2263/drum_data/val/'+'J_'+str(J)+'Q_'+str(Q)+'order_'+str(order)+'.pkl', 'wb')
pickle.dump([val_input,val_gt], output)
output.close()